In [10]:
import arcpy
import os
import sys
import fnmatch
import itertools
import datetime
from math import  *
import numpy
import gdal
import xml.etree.ElementTree as et

## L = Gain x DN x (abscalfactor/effective bandwidth) + Offset  ------Equation 1 Spectral Radiance
## r = (pi * L(band) * dES^2) / (Esun * Cos(90 - meanSulEl)) ---------Equation 2 Planetary Reflecatance

Link to the Maxar document [here](https://cdn1-originals.webdamdb.com/13264_102978826?cache=1581093759&response-content-disposition=inline%3Bfilename%3D30004_arc_202001.pdf.pdf&response-content-type=application%2Fpdf&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cCo6Ly9jZG4xLW9yaWdpbmFscy53ZWJkYW1kYi5jb20vMTMyNjRfMTAyOTc4ODI2P2NhY2hlPTE1ODEwOTM3NTkmcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj1pbmxpbmUlM0JmaWxlbmFtZSUzRDMwMDA0X2FyY18yMDIwMDEucGRmLnBkZiZyZXNwb25zZS1jb250ZW50LXR5cGU9YXBwbGljYXRpb24lMkZwZGYiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjIxNDc0MTQ0MDB9fX1dfQ__&Signature=C0jVjal-2zWoGAivWwcl2-Ob5s8V-VtudJn6hhN96gFkloZObxc20rfdhEJIB~BxZGQ~zwBMjFBKvPZifB~yVZD5eFbXyutPf9AQV1hHNCwoXjPwRqVbjaEtxxHw6QMa8Cy9f5FO3L2A-1snDC1KLfFEqK66uBXL7tNAgDhAkkX6pUfVmcVJwe46JNlzKgvM5hugxmn6BzYrBq2VSh8cadGLYBT4umu3GpdSN1Bbl9L8lVqQtF7kiIYpdazCcPloWvWUoLvw1IvVh8jV~B4Qvau-wMZ~G9zFRjidhn1sXQj29EcroN-e0oxZiu6Ujgyo3HH2~Au1NKV1edzkcFjQVQ__&Key-Pair-Id=APKAI2ASI2IOLRFF2RHA)
 and [this](https://dg-cms-uploads-production.s3.amazonaws.com/uploads/document/file/209/ABSRADCAL_FLEET_2016v0_Rel20170606.pdf) - 2016v

In [11]:
#constant parameters 
pi = 3.14159265358
# Extracted from Absolute Radiometric Calibration: 2016v
offset = {'Band_1': -5.736, 'Band_2': -3.546,'Band_3': -2.512,  'Band_4': -3.300}
gain = {'Band_1':0.988, 'Band_2': 0.36, 'Band_3':0.952, 'Band_4':0.961}
esun = {'Band_1':2007.27, 'Band_2':1829.62,'Band_3': 1538.85, 'Band_4':1053.21}


In [12]:
folder = r'C:\\Toar\\NGA_Ngala_WV2_12Aug2016_2016_55_249_original\\055795286010_01\\055795286010_01_P001_MUL\\' # path to the P001_MUL (The MultiSpectral folder)

#get image and metadata paths
def find_paths(folder):
    image =[]
    metadata_path =[]  
    #arcpy.env.workspace = r'C:\Toar'= wkspc
    for k in os.listdir(folder):
        if k.endswith('.XML'):
            metadata_path = os.path.join(folder,k)
        if k.endswith('.TIF'):
            image = os.path.join(folder,k)
    return(metadata_path, image)

m_path, i_path = find_paths(folder)

print(m_path)
print(i_path)

C:\\Toar\\NGA_Ngala_WV2_12Aug2016_2016_55_249_original\\055795286010_01\\055795286010_01_P001_MUL\\16AUG12093434-M2AS-055795286010_01_P001.XML
C:\\Toar\\NGA_Ngala_WV2_12Aug2016_2016_55_249_original\\055795286010_01\\055795286010_01_P001_MUL\\16AUG12093434-M2AS-055795286010_01_P001.TIF


In [13]:
## METADATA
#read neccesary parameters from the metadata
def find_metad(metadata_path):
    #iterate through the metada
    tree = et.parse(metadata_path)
    root = tree.getroot()

    #uncomment to see the tag for each band (but not neccesary)
    # [elem.tag for elem in root.iter()]

    #Time
    for d in root.iter('GENERATIONTIME'):
        dt = d.text #dt = d.text[:10]
    #Sun elevation Angle
    for a in root.iter('IMAGE'):
        for i in a.iter('MEANSUNEL'):
            sza = 90 - float(i.text) #sun elevation angle converted to sun zenith angle

    # extract the abscalfactor and effectivebandwidth 
    band_names = ['BAND_B', 'BAND_G','BAND_R', 'BAND_N'] #As written in the metadata
    g=[]
    for name in band_names:
        for b in root.iter(name):
            for b,c in itertools.product(b.iter('EFFECTIVEBANDWIDTH'), b.iter('ABSCALFACTOR')):
                d =float(c.text)/float(b.text) #(abscalfacto/effective bandwidth) --- see equation 1
                d = round(d,7)
                g.append(d)
    dic = dict(zip(band_names,g))
    rnm = {'BAND_B': 'Band_1', 'BAND_G': 'Band_2','BAND_R': 'Band_3',  'BAND_N': 'Band_4'} #Change band name to numbers
    band_factors = dict((rnm[key], value)for (key, value) in dic.items())
    return(band_factors, sza, dt)

absl, sza, dt  = find_metad(m_path)
# print(absl, sza, dt)

In [14]:
# Get Julian day --Need review
#https://radixpro.com/a4a-start/julian-day-and-julian-day-number/
def datestdtojd(dt):
    fmt='%Y-%m-%dT%H:%M:%S.%fZ'
    dt = datetime.datetime.strptime(dt, fmt)
    dt = dt.timetuple()
    yr = int(dt.tm_year)
    mon = int(dt.tm_mon)
    dy = int(dt.tm_mday)
    if mon in (1,2):
        yr -= 1
        mon += 12
    # doy = dt.tm_yday
    hr = int(dt.tm_hour)
    mit = int(dt.tm_min)/60.
    sec = float(float(dt.tm_sec)/3600.)
    tm = (hr+mit+sec)/24.0
    
    A = int(yr/100)                     
    B = 2- A + int(A/4)

    jd = int(365.25 * (yr + 4716)) + \
        int(30.6001 * (mon + 1)) + \
        dy + tm +\
        B - 1525.5
    
    """GRASS GIS i.toar"""
    D = jd - 2451545.0
    g = 357.529 + 0.98560028 * D
    gr = math.radians(g)
    dES = 1.00014 - 0.01671 * math.cos(gr) - 0.00014 * math.cos(2 * gr)
    return dES

jdt = datestdtojd(dt)
jdt

1.0003964721944973

In [19]:
#output path 
Prod_D = os.path.join(folder, 'Prod_D')
if not os.path.exists(Prod_D):
    os.mkdir(Prod_D)
#output file name
outfile =  os.path.join(Prod_D,'processed_calrefbyt_vhr.dat')

# copy raster properties from input raster
Ds = gdal.Open(i_path, gdal.GA_ReadOnly)
ysize = Ds.RasterYSize
xsize = Ds.RasterXSize
transf = Ds.GetGeoTransform()
proj = Ds.GetProjection()
#output format
driver = gdal.GetDriverByName('ENVI')   
#
bnum, bnums = 1,4
output_file = driver.Create(outfile, xsize,ysize, bnums, gdal.GDT_Byte)
output_file.SetGeoTransform(transf)
output_file.SetProjection(proj)

for i in range(1,5):
    data = None
    inDs = gdal.Open(i_path, gdal.GA_ReadOnly)
    band = inDs.GetRasterBand(i)
    data = band.ReadAsArray().astype(numpy.float16)
    data[data ==0] = numpy.nan
    band.FlushCache()
    band = None
    
    if i in range(1,5):
        #Get parameters for each band
        gn = float(gain['Band_'+str(i)])
        asf = float(absl['Band_'+str(i)])
        oft = float(offset['Band_'+str(i)])
        esn = float(esun['Band_'+str(i)]) 
        # cal. TOA 
        data = gn * data * asf + oft # -- Equation 1
        data = (pi * data * jdt**2) / (esn * math.cos(math.radians(sza))) # -- Equation 2
        data[data<0] = 0
        data[data>1] = 1
        data *=255

        #write bands in a file .dat
        refloutp = output_file.GetRasterBand(bnum)
        refloutp.WriteArray(data.astype(numpy.float32))
        refloutp.FlushCache()
        refloutp = None
        bnum += 1

#close files
data = None
inDs = None
outfile = None
output_file = None
refloutp = None


In [7]:
# arcpy.env.workspace = i_path
# proc_path = r'C:\Toar\Proc\\'
# Prod_D = os.path.join(folder, 'Prod_D')
# if not os.path.exists(Prod_D):
#     os.mkdir(Prod_D)
# comp_file = os.path.join(Prod_D, 'Composite.tif')
# dat_file = os.path.join(Prod_D, 'datrfk.dat')
# def cal_radiance(path_):
#     bands = [Raster(os.path.join(i_path, b)) for b in arcpy.ListRasters()]

#     for band in bands:
#         bn = str(band.name)
#         band = Raster(band)   
#         bradiance = gain[bn] * band * absl[bn] + offset[bn]
#         refl = (pi * bradiance * jdt**2) / (esun[bn] * math.cos(math.radians(sza)))
#         # dd = esun[bn] * math.cos(sunel)
#         # refl = hg/dd 
#         # refl.save(proc_path+bn+'_calfel.tif')
#     for i in arcpy.ListRasters():
#         arcpy.CompositeBands_management(i, comp_file)
#         arcpy.CopyRaster_management(comp_file, dat_file, '','', '','','','8_BIT_UNSIGNED','','','ENVI')     
#     return(refl) 

# # def cal_reflectance(bradiance):
# # arcpy.CopyRaster_management()
